In [1]:
require './market.rb'
require './script/helpers.rb'

true

In [2]:
tids  = Commodities.map {|t| t.id }
debut = Time.parse('1 jan 2019')
fin   = Time.parse('31 dec 2019')
bars  = Bar.where(:date => debut..fin, :ticker_id => tids)
           .order(:ticker_id, Sequel.asc(:date))
           .all
groups  = bars.group_by {|b| b.ticker_id }
changes = groups.map do |ticker_id, bars|
  bars.each_cons(2).map {|a, b| b.change_from a }
end.flatten

# This will sort the changes into the bucket that are closest in value
hist = changes.histogram

[[-0.16057713963963965, -0.15721218814968815, -0.15384723665973665, -0.15048228516978518, -0.14711733367983368, -0.1437523821898822, -0.1403874306999307, -0.13702247920997923, -0.13365752772002773, -0.13029257623007623, -0.12692762474012476, -0.12356267325017325, -0.12019772176022177, -0.11683277027027028, -0.1134678187803188, -0.1101028672903673, -0.1067379158004158, -0.10337296431046432, -0.10000801282051283, -0.09664306133056134, -0.09327810984060986, -0.08991315835065836, -0.08654820686070687, -0.08318325537075538, -0.0798183038808039, -0.07645335239085241, -0.07308840090090092, -0.06972344941094942, -0.06635849792099793, -0.06299354643104645, -0.05962859494109496, -0.05626364345114347, -0.05289869196119197, -0.049533740471240484, -0.046168788981289, -0.04280383749133751, -0.03943888600138602, -0.036073934511434536, -0.032708983021483035, -0.02934403153153156, -0.02597908004158006, -0.02261412855162856, -0.019249177061677086, -0.015884225571725585, -0.012519274081774112, -0.0091543

In [3]:
h = hist[0].zip(hist[1])
h.sort_by {|x, y| y }[-20..-1].each {|xy| puts xy }

[-0.032708983021483035, 41.0]
[-0.02934403153153156, 49.0]
[0.03122509528759526, 51.0]
[0.027860143797643788, 61.0]
[-0.02597908004158006, 72.0]
[0.024495192307692287, 85.0]
[-0.02261412855162856, 102.0]
[-0.019249177061677086, 122.0]
[0.021130240817740786, 124.0]
[0.017765289327789313, 164.0]
[0.014400337837837812, 177.0]
[-0.015884225571725585, 194.0]
[0.011035386347886339, 232.0]
[-0.012519274081774112, 234.0]
[-0.009154322591822611, 303.0]
[0.007670434857934838, 313.0]
[-0.00578937110187111, 402.0]
[0.004305483367983337, 530.0]
[-0.002424419611919637, 680.0]
[0.0009405318780318639, 1195.0]


[[-0.032708983021483035, 41.0], [-0.02934403153153156, 49.0], [0.03122509528759526, 51.0], [0.027860143797643788, 61.0], [-0.02597908004158006, 72.0], [0.024495192307692287, 85.0], [-0.02261412855162856, 102.0], [-0.019249177061677086, 122.0], [0.021130240817740786, 124.0], [0.017765289327789313, 164.0], [0.014400337837837812, 177.0], [-0.015884225571725585, 194.0], [0.011035386347886339, 232.0], [-0.012519274081774112, 234.0], [-0.009154322591822611, 303.0], [0.007670434857934838, 313.0], [-0.00578937110187111, 402.0], [0.004305483367983337, 530.0], [-0.002424419611919637, 680.0], [0.0009405318780318639, 1195.0]]

In [68]:
buy_signals = buys :stocks => Commodities, :year => 2019, :drop => -0.15
buy_signals.size

2

In [69]:
buy_signals.map do |bar|
  bar.max_rise_over(30)[1]
end.mean

0.14172960518929967

In [70]:
buy_signals.map {|b| b.rank }

[19, 19]

In [71]:
ass = Assessor.new
ass.sell_when do |original, today|
  today.change_from(original) >= 0.15
end

ass.holding = buy_signals
res = ass.assess_sells
res.size

2

In [73]:
prof = profit res, :pieces => 2, :debug => false, :reinvest => true

puts "$#{res.map {|r| r[:buy].open }.mean}"
puts res.map {|r| r[:ROI] }.mean
puts "#{res.map {|r| r[:hold] || 1000 }.median} days"
puts "#{prof[:skips].size} skips"
puts "$15.0 -> $#{prof[:cash]}"

$348.375
0.1589524276434437
276 days
0 skips
$15.0 -> $17.384286414651655


In [74]:
Split.first

#<Split @values={:id=>1, :ticker_id=>8, :ratio=>"103 for 100\n", :date=>1962-12-18 00:00:00 -0500, :applied=>nil}>

In [75]:
Split.all.count {|s| s.applied }

626

In [76]:
Split.count

2771

In [77]:
Split.where(:ticker_id => NYSE.map {|t| t.id }).count

2771